Data Modeling Practice
----------------------

In this exercise, you will demonstrate your understanding of normalization by designing a data model. You have two files: `https://s3-us-west-2.amazonaws.com/dsci6007/data/dbip-city-2015-07.csv` (henceforth `dbip`) and `https://s3-us-west-2.amazonaws.com/dsci6007/data/link.log`.  
`dbip` is essentially a lookup table to find the geolocation for any given IP address. However, because there are over four billion IPv4 addresses (to say nothing of IPv6), each row represents a range. For example:

    ip_start     	ip_end     	country 	stateprov   	city
    194.45.112.64   194.45.112.71   NL     Gelderland     Arnhem
    194.45.112.72   194.45.112.79   NL        Drenthe  Hoogeveen
    194.45.112.80   194.45.112.87   NL  North Holland  Amsterdam
    194.45.112.88   194.45.112.95   NL  North Holland     Diemen
    194.45.112.96   194.45.112.103  NL  South Holland  Rotterdam
    194.45.112.104  194.45.112.111  DE      Thuringia     Erfurt
    194.45.112.112  194.45.112.119  NL  South Holland  Rotterdam
    194.45.112.120  194.45.112.135  DE      Thuringia     Erfurt

`link.log` contains simulated data representing a link from one IP address to one or more other IP addresses. The data look like this:

    source         	trusted	ts              	destinations
    49.73.120.173   False   2015-07-07 13:13:29 {"40.158.102.3"}
    206.248.235.182 False   2015-07-07 13:13:32 {"33.159.193.158"}
    186.39.249.152  False   2015-07-07 13:13:33 {"206.208.143.6", "96.168.100.43"}

    
What we want is a fully normalized database supporting SQL queries that will be able to show us which cities linked to which, at what time, and so on.

1. Create a normalized database schema for these data using Gliffy.
2. Write `CREATE TABLE` statements to generate these tables.

Normalization Practice
----------------------

Starting from the solution to this previous exercise, work in pairs to load the data into your database. Then create views to answer the following questions:

1. For each country, how many trusted and how many untrusted incoming links are made?
2. For each country, how many domestic and how many international outgoing links are made?


 
 
 
 
1. For each country, how many trusted and how many untrusted incoming links are made?

NOTE: STILL NEED TO DRAW A VEN DIAGRAM TO FULL UNDERSTAND HOW THIS QUERY WORKS!!!

        SELECT dbip.country,trust.trusted,  COUNT(trust.trusted )  
        FROM link 
        JOIN dbip 
        ON (link.source >= dbip.ip_start AND link.source <= dbip.ip_end)  
        JOIN trust 
        ON (link.source = trust.source ) 
        GROUP BY dbip.country, trust.trusted   
        LIMIT 10;
        
         country | trusted | count 
        ---------+---------+-------
         KR      | t       |  1448
         RS      | t       |    17
         VN      | t       |   226
         AR      | f       |   248
         PK      | f       |    55
         MD      | f       |    41
         NA      | t       |    15
         SV      | t       |    13
         QA      | t       |    21
         TZ      | f       |    25
        (10 rows)
        
As a check: 

        SELECT dbip.country,trust.trusted,  COUNT(trust.trusted )  
        FROM link 
        JOIN dbip 
        ON (link.source >= dbip.ip_start AND link.source <= dbip.ip_end )  
        JOIN trust 
        ON (link.source = trust.source ) 
        WHERE dbip.country = 'US'    
        GROUPBY dbip.country  , trust.trusted;
        
         country | trusted | count 
        ---------+---------+-------
         US      | t       | 25766
         US      | f       | 26131
        (2 rows)

2 For each country, how many domestic and how many international outgoing links are made?

    Groups ip_start and ip_end by country names:  

    create view country_view as
    select dbip.country  as country,
           dbip.ip_start as start,
           dbip.ip_end as end 
    from dbip
    order by dbip.country;

    Tags ip_source with country names:

    create view outgoing_view as 
    select link.source, dbip.country as outgoing
    from link 
    join dbip 
    on (link.source >= dbip.ip_start AND link.source <= dbip.ip_end) 
    group by outgoing,link.source; 


    Tags ip_destinatin with country names:

    create view incoming_view as
    select link.destination, country_view.country as incoming
    from link 
    join country_view
    on (link.destination >= country_view.start AND link.destination <= country_view.end) 
    order by incoming, link.destination;


    Briges outgoing and incoming class:
    create view outgoing_incoming as
    select outgoing_view.outgoing, incoming_view.incoming
    from outgoing_view
    left join link
    on (outgoing_view.source = link.source)
    right join incoming_view
    on (incoming_view.destination = link.destination)
    order by outgoing_view.outgoing, incoming_view.incoming;


    select country_view.country as outgoing, link.source
    from link 
    left join country_view
    on (link.source >= country_view.start AND link.source <= country_view.end) 
    order by outgoing, link.source
    limit 10; 


    select outgoing_view.country, link.source
    from link
    left join outgoing_view
    on (link.so)


    Gets country initials for source ip addresses: 

    create view outgoing_view as 
    select link.source, dbip.country as outgoing
    from link 
    join dbip 
    on (link.source >= dbip.ip_start AND link.source <= dbip.ip_end) 
    group by outgoing,link.source; 


    Gets country initials for destination ip addresses: 

    select link.destination, dbip.country as incoming
    from link 
    join dbip 
    on (link.destination >= dbip.ip_start AND link.destination <= dbip.ip_end) 
    order by link.destination 
    limit 10;


    Get's bridge table:

    select link.source, link.destination
    from link 
    order by link.source, link.destination
    limit 10; 


    select link.source as right_source, dbip.country as outgoing, dest

    from link 
    join dbip 
    on (link.source >= dbip.ip_start AND link.source <= dbip.ip_end)

    left join

            (
            select link.source as left_source, link.destination as dest
            from link 
            order by link.source   
            ) as table

    on (right_source = left_source)

    order by outgoing, dest
    limit 10;





    _____________
    select outgoing, link.destination

    from link,
    (
    select link.source, dbip.country as outgoing
    from link 
    join dbip 
    on (link.source >= dbip.ip_start AND link.source <= dbip.ip_end) 
    order by link.source 
     )  as left

    join

    (
    select link.source, link.destination
    from link  
    ) as  right

    on (left.source = right.source)
    group by outgoing
    limit 10;


    __________
    select link.source, table.incoming,



    (
    select link.destination as dest , dbip.country as incoming
    from link 
    join dbip 
    on (link.destination >= dbip.ip_start AND link.destination <= dbip.ip_end) 
    order by link.destination 
    ) as table

    from link 
    join table
    on (link.destination = table.dest)

    group by link.source
    limit 10;


select dbip.



    create table dbip (
        ip_start  cidr PRIMARY KEY,
        ip_end    cidr NOT NUll,
        country   varchar(2) NOT NULL,
        stateprov varchar(500) NOT NULL,
        city      varchar(500) NOT NULL 
        );
    
    
    create table link (
        source cidr references trust(source),
        ts     timestamp NOT NULL,
        destination cidr NOT NULL, 
        PRIMARY KEY(source, ts, destination)
        );


    create table trust (
        source cidr PRIMARY KEY, 
        trusted boolean NOT NULL
        );
    
    copy dbip from '/Users/Alexander/dbip-city-2015-07.csv' delimiter ',' csv;
    
    
        copy temp from '/Users/Alexander/link.log';
        
        
    create table temp (
        source cidr NOT NULL,
        trusted boolean NOT NULL,
        ts      timestamp NOT NULL,
        destination varchar(1000) NOT NULL,
        PRIMARY KEY(source, ts, destination)
        );


        select source, trusted, ts, unnest(temp.destination::text[]) from limit 4;
        
        insert into link (source, ts, destination) select temp.source, temp.ts, unnest(temp.destination::text[])::cidr from  temp;
        
    insert into trust ( source, trusted) select  distinct temp.source, temp.trusted from temp;
    
    1) loaded link.log into temp table 
    2) unnest destinations 
    3) change data type of unnested destinations from text to cidr
    4) insert relevent columns into link and trust tables
    
 
 